# Laplace approximation

## Quick recap
We started with our simple question: 'Is there a relationship between the flipper length of penguin and their weights'. We saw that we can use the linear model to answer that question based on some data. We also need the Bayes theorem to be able to calculate the posterior, which will inform us of the likelihood of the values of the parameters of our model given the data. And if, given our data, the value that characterize the relationship between flipper length and weight is unlikely to be close to 0, then we can answer our question with some amount of confidence.

In order to apply the Bayes theorem to our linear model, we need to multiply the likelihood of our data given any values of the parameters ($P(y|\Theta)$) by our prior ($P(\Theta)$), which characterizes our belief about likelily values of the parameters of our model. Then, we need to divide that whole thing by a scaling constant: the marginal likelihood or the model evidence, which is basically the integration of the numerator ($P(y|\Theta)  * P(\Theta)$) over all possible values of $\Theta$. And as we saw, in the case of our model, the likelihood is a multivariate distribution, while the prior consists of the product between a normal distribution for the $\beta$ parameters of our model and a Delta distribution for the $\sigma^2$ parameter of our model. And unfortunately, with this combination of distributions, we can't find an easy (i.e. analytical) solution for the integral in the denominator.

In other words, for our simple model, we can't compute the model evidence. In the previous chapter, we found a way to circumvent that by relying on the Jensen inequality. Instead of directly computing the model evidence, we can instead optimize the free energy function, which is defined as:

$$F[Q(\Theta)] = \mathbb{E}_{Q(\Theta)}[lnP(y, \Theta) - ln Q(\Theta)]$$

So we have to find the $Q(\Theta)$ distribution with which the output of the free energy functional is maximal, in which case that output is the **free energy**, which is an approximation of the log of the marginal likelihood. And also, the distribution $Q(\Theta)$ is maximal is itself an approximation of the posterior.

So now, in order to answer our question 'Is there a relationship between the flipper length of penguin and their weights', our goal is to maximize the free energy functional, which will yield the model evidence and the posterior so that we can get our confidence for the true value of our model parameter to be able to answer our question. In other words, what we have to do conceptually is try out a lot of different possible $Q(\Theta)$ distribution until we find the one that yields the largest value of $F[Q(\Theta)]$.

Yeah? All good? Perfect, now let's get to it.

## Dealing with the integral

To compute the free energy, we still need to deal with integrals. Indeed, the E symbol hides an integral over all values of $\Theta$. We can actually rewrite it like so:

$$F[Q(\Theta)] = \int_{-\infty}^{+\infty}[ln P(y, \Theta) - ln Q(\Theta)] d(\Theta)$$

One thing we can do to make the free energy functional a bit easier to deal with is this:

$$F[Q(\Theta)] = \mathbb{E}[ln P(y, \Theta)] - \mathbb{E}[ln Q(\Theta)]$$

That doesn't solve anything. But now, we can deal with the Expectation of the log of the joint probability $P(y, \Theta)$ and the expectation of the log of the approximate posterior $Q(\Theta)$ separately, meaning if we can figure each part out, then we can simply take the substraction of the latter with the former. 

But how do we solve the integral in each of the part? This is where the Laplace comes in. Basically, everything that comes below is to find a way to get rid of the integral in each of the parts of the free energy functional. The Laplacian method will enable us to approximate the result of the integral without actually having to calculate it. In this particular instance, you can actually skip most of the math and go right to the end, where I will provide the formula for the free energy without any integral. With that, you could go straight ahead and actually implement a function to solve it. I of course encourage you nonetheless to at least read the math once, so that you have coarse idea of how we did it.

## Quadratic approximation
The maths discussed in the previous chapter was perhaps  a bit difficult to wrap your head around from a conceptual standpoint, because it was a bit difficult to see where it was leading to. But at least each single bit was rather simple mathematic, the expectation being a general formula of something you have used many times. So not too complicated mathematic, just a bit complicated to motivate at first. 

In this chapter, it is a bit the opposite: what need to do is really clear (get rid of the integral), but the math behind it are unfortunately more complicated than what we saw before. The first piece of that puzzle is the so-called Quadratic approximation. 

Depending on your background, you may already have heard of "Taylor series expansion". This is a mathematical concept that says that you can approximate any function around a particular point by taking the sum of the derivatives of that function. In other words, for any function for which you can actually compute many derivatives, you can approximate it around a point $x_0$ like so:

$$f(x) \approx f(x_0) + \frac{f'(x_0)}{1!}(x-x_0) + \frac{f''(x_0)}{2!}(x-x_0)^2 + \frac{f'''(x_0)}{3!}(x-x_0)^3 ...$$

The quadratic approximation is a Taylor series expansion where you stop at the second derivative basically:

So we say that:

$f(x_0) + \frac{f'(x_0)}{1!}(x-x_0) + \frac{f''(x_0)}{2!}(x-x_0)^2$

Is the quadratic approximation of $f(x)$

We can write it a bit more simply:

$$f(x) \approx f(x_0) + f'(x_0)(x-x_0) + \frac{1}{2}f''(x_0)(x-x_0)^2 $$

You are probably familiar with the concept of a derivative of the first and second order. If you have a function:

$f(x) = x^2$

The the first order derivative of that function is:

$f'(x) = 2x$

And the second order derivative is the derivative of the derivative:

$f''(x) = 2$

And you might also remember that the first order derivative defines the slope of the tangent of function at any given point. So for our function $f(x) = x^2$, the slope at say $x=2.5$ is $f'(2.5)=2*2.5=5$. In other words, the derivative is the rate of a change of a function at a single point. Hopefully, that should already be familiar. The second order derivative perhaps a bit less, but it defines the curvature of a function at a particular point. What that exaclty means isn't very important right now, this is just a very quick refresher about what derivatives are.

Now one thing you might be less familiar with (at least I was before diving into this) is what happens to the concept of derivatives when we are dealing with functions that don't take single values as inputs, but rather vectors. The whole reason why we are talking about any of this is because ultimately, we want to approximate the expectation of $Q(\Theta)$ and $P(y, \Theta)$. But $\Theta$ isn't a single value, it is a vector that contains a value for each parameter of our model. In the case of vectors, the first order derivative of a function is something we call a gradient. And the second order derivative is something we call a Hessian matrix. We won't have time to dig into this much further here, but if you want to know more about each, I highly recommend [this series](https://www.youtube.com/@digitallearninghub-imperia3540) of videos on youtube, which are explaining multivariate calculus very well.

In any case, when dealing with functions that take vectors as their input, the quadratic approximate becomes this:

$$f(x) \approx f(x_0) + \nabla f(x_0)^T(x-x_0) + \frac{1}{2}(x-x_0)^T H_f(x_0)(x-x_0)$$

Where:
- $\nabla f(x_0)$: is the gradient (i.e. the first order derivative of a vector function) of $f(x)$
- $H_f(x_0)$ is the Hessian matrix at $x_0$

This may look a bit intimidating, and you might not know how to compute each of the parts. But we don't need to worry about it just now, as parts of the complicated bits will disappear as we move along

## Quadratic appproximation of $Q(\Theta)$

As we saw in the previous chapter, we don't really know what kind of a distribution $ln Q(\Theta)$ is, we just know that it is a probability distribution. So it is one of the function that we want to approximate in order to be able to calculate the free energy. This may sound very confusing: I said before that the problem of the free energy functional are the integrals, not $ln Q(\Theta)$. And while it is true that $Q(\Theta)$ is not know and needs to be figured out, it is not what we set out to do, is it? Well as you will see down the line, we are kind of tackling both at the same time: **we approximate $Q(\Theta)$ with something else for which we can compute the integral analytically!** 

You might wonder: why not apply the quadratic approximation to $E_{Q(\Theta)}[ln Q(\Theta)]$? The answer is simply that we can't

So let's write the quadratic approximation of $ln Q(\Theta)$:

$$ln Q(\Theta) \approx ln Q(\Theta) + \nabla ln Q(\Theta)^T(\Theta-\Theta_0) + \frac{1}{2}(\Theta-\Theta_0)^T H_Q(\Theta_0)(\Theta-\Theta_0)$$
